In [96]:
import datetime as dt
import pandas as pd
import numpy as np
import math
from functools import reduce

In [97]:
# Set Peak
peak_month = [3,4,5,6,7,8,9,10,11,12,1,2]
peak_arr = [95,95,94,90,91,72,66,87,91,85,100,95]

peak_year = 2020
peak_date = list()
peak_df = pd.DataFrame(columns=["peak (kW)"])

for idx,_ in enumerate(peak_month):
    peak_date.append(
        dt.datetime(
            peak_year,
            _,
            1
        )
    )
    peak_df = peak_df.append({
        "peak (kW)": peak_arr[idx] 
    },ignore_index=True)
    
peak_df.index = peak_date
peak_df

,peak (kW)
2020-03-01,95
2020-04-01,95
2020-05-01,94
2020-06-01,90
2020-07-01,91
2020-08-01,72
2020-09-01,66
2020-10-01,87
2020-11-01,91
2020-12-01,85


In [98]:
# set household
household_name = ["{}01 호".format(_) for _ in range(1, 11)]
household_kwh = [150, 180, 220, 210, 310, 300, 270, 190, 250, 260]
household_df = pd.DataFrame(columns=[
    "name",
    "usage (kWh)"
])

for idx, kwh in enumerate(household_kwh):
    household_df = household_df.append({
        "name": household_name[idx],
        "usage (kWh)": kwh
    }, ignore_index=True)

household_df

,name,usage (kWh)
0,101 호,150
1,201 호,180
2,301 호,220
3,401 호,210
4,501 호,310
5,601 호,300
6,701 호,270
7,801 호,190
8,901 호,250
9,1001 호,260


# 요금 설정

In [99]:
# set kepco fee
KEPCO_FEE_STATIC = {
    "환경비용차감": -5,
    "기후환경요금": 5.3,
    "연료비조정액": -3,
    "필수사용량 보장공제 (월 1,000 kWh 초과), 저압": 709.5,
    "필수사용량 보장공제 (월 1,000 kWh 초과), 고압": 574.6,
}
KEPCO_FEE_DF = pd.DataFrame(columns=['name', 'fee', 'unit'])
for key in KEPCO_FEE_STATIC.keys():
    KEPCO_FEE_DF = KEPCO_FEE_DF.append({
        "name": key,
        "fee": KEPCO_FEE_STATIC[key],
        "unit": 'kWh'
    }, ignore_index=True)
    
KEPCO_FEE_STATIC_II = {
    "필수사용량 보장공제 (월 200 kWh 이하), 저압": 4000,
    "필수사용량 보장공제 (월 200 kWh 이하), 고압": 2500,
}
for key in KEPCO_FEE_STATIC_II.keys():
    KEPCO_FEE_DF = KEPCO_FEE_DF.append({
        "name": key,
        "fee": KEPCO_FEE_STATIC_II[key],
        "unit": None
    }, ignore_index=True)

KEPCO_FEE_DF

,name,fee,unit
0,환경비용차감,-5,kWh
1,기후환경요금,5.3,kWh
2,연료비조정액,-3,kWh
3,"필수사용량 보장공제 (월 1,000 kWh 초과), 저압",709.5,kWh
4,"필수사용량 보장공제 (월 1,000 kWh 초과), 고압",574.6,kWh
5,"필수사용량 보장공제 (월 200 kWh 이하), 저압",4000,None
6,"필수사용량 보장공제 (월 200 kWh 이하), 고압",2500,None


In [100]:
# 주택용전력 (저압) setting
fee_dict = dict()

household_low_pressure_fee = {
    "type":["주택용 저압" for _ in range(0,12)],
    "max kWh": reduce(lambda acc, cur: acc + [200,400,float("inf")], range(0,3),[]) \
        + [300, 450, float("inf")],
    "basic": reduce(lambda acc, cur: acc + [910, 1600, 7300], range(0,4),[]),
    "fee": reduce(lambda acc, cur: acc + [88.3, 182.9, 275.6], range(0,4),[]),
    "unit":["kWh" for _ in range(0,12)],
    "season":["spring" for _ in range(0,3)] + ["autumn" for _ in range(0,3)] + \
        ["winter" for _ in range(0,3)] + ["summer" for _ in range(0,3)]
}

household_low_pressure_fee_df = pd.DataFrame(household_low_pressure_fee,
                      columns=['type','max kWh', 'basic', 'fee', 'unit','season'])

fee_dict['주택용전력 (저압)'] = household_low_pressure_fee_df

In [101]:
# 일반용전력(갑) 1 setting
public_fee = {
    "type":["일반용전력(갑) 1, 저압" for _ in range(0,4)] +\
        ["일반용전력(갑) 1, 고압A, 선택 1" for _ in range(0,4)] +\
        ["일반용전력(갑) 1, 고압A, 선택 2" for _ in range(0,4)] +\
        ["일반용전력(갑) 1, 고압B, 선택 1" for _ in range(0,4)] +\
        ["일반용전력(갑) 1, 고압B, 선택 2" for _ in range(0,4)],
    "max kWh": [float("inf") for _ in range(0,20)],
    "basic": reduce(lambda acc, cur: acc + [cur for _ in range(0,4)],
                    [6160,7170,8230,7170,8230],[]),
    "fee": [60.2, 100.7, 60.2, 87.3] +\
        [66.9, 110.9, 66.9, 98.6] +\
        [62.6, 106.9, 62.6, 93.3] +\
        [65.8, 108.9, 65.8, 95.6] +\
        [60.5, 103.5, 60.5, 90.3],
    "unit":["kWh" for _ in range(0,20)],
    "season": reduce(lambda acc, cur: acc + ["spring", "summer", "autumn", "winter"],
                    range(0,5), [])
}

public_fee_df = pd.DataFrame(public_fee,
                      columns=['type','max kWh', 'basic', 'fee', 'unit','season'])

fee_dict['일반용전력(갑) 1'] = public_fee_df

In [102]:
# 전체통합
fee_df = pd.DataFrame(columns=['type','max kWh', 'basic', 'fee', 'unit','season'])
for _ in fee_dict.values():
    fee_df = pd.concat([fee_df, _], ignore_index=True)
    
fee_df

,type,max kWh,basic,fee,unit,season
0,주택용 저압,200.0,910,88.3,kWh,spring
1,주택용 저압,400.0,1600,182.9,kWh,spring
2,주택용 저압,inf,7300,275.6,kWh,spring
3,주택용 저압,200.0,910,88.3,kWh,autumn
4,주택용 저압,400.0,1600,182.9,kWh,autumn
5,주택용 저압,inf,7300,275.6,kWh,autumn
6,주택용 저압,200.0,910,88.3,kWh,winter
7,주택용 저압,400.0,1600,182.9,kWh,winter
8,주택용 저압,inf,7300,275.6,kWh,winter
9,주택용 저압,300.0,910,88.3,kWh,summer


# 종합계약
- 세대부 (household) : 주택용전력(저압)
- 공용부 (public) : 일반용전력(갑) 1 선택 2

## 세대부
- Season : winter
- Calc Month : 2

In [103]:
# 세대부 요금 계산
bill_df = pd.DataFrame(columns=["기본요금","전력량요금","기후환경요금","연료비조정액","필수사용량 보장공제"])

household = household_df.iloc[0]
sel_fee_df = fee_df[
    (fee_df['type'] == "주택용 저압") &\
    (fee_df['season'] == 'winter')
].copy()

print("#################################################")
print("세대명 : {}".format(household['name']))
fees = list()
fee_dict = dict()

# 기본요금
basic_fee = sel_fee_df[
    sel_fee_df['max kWh'] >= household['usage (kWh)']
]['basic'].iloc[0]
basic_fee
fees.append(basic_fee)
fee_dict['기본요금'] = basic_fee
print("기본요금 : {}원".format(basic_fee))

# 전력량요금
electricity_rate = 0
fix_usage = household['usage (kWh)']
usage = household['usage (kWh)']
step_bak = 0
idx = 0
while usage > 0:
    progressive_tax = sel_fee_df['fee'].iloc[idx]
    max_kwh = sel_fee_df['max kWh'].iloc[idx]

    step_kwh = usage if usage < max_kwh else max_kwh - step_bak
    electricity_rate += int(progressive_tax * step_kwh)
    
    usage -= step_kwh
    step_bak = max_kwh
    idx += 1
fees.append(electricity_rate)
fee_dict['전력량요금'] = electricity_rate
print("전력량요금 : {}원".format(electricity_rate))

# 기후환경요금
unit = KEPCO_FEE_STATIC['기후환경요금']
env_fee = int(fix_usage * unit)
fees.append(env_fee)
fee_dict['기후환경요금'] = env_fee
print("기후환경요금 : {}원".format(env_fee))

# 연료비조정액
unit = KEPCO_FEE_STATIC['연료비조정액']
fuel_cost_adjustment = int(fix_usage * unit)
fees.append(fuel_cost_adjustment)
fee_dict['연료비조정액'] = fuel_cost_adjustment
print("연료비조정액 : {}원".format(fuel_cost_adjustment))

# 필수사용량 보장공제 여부
is_deduction = True if (fix_usage > 1000) \
                or (fix_usage <= 200) else False
print("필수사용량 보장공제 여부 : {}".format(is_deduction))
if is_deduction:
    unit = KEPCO_FEE_STATIC['필수사용량 보장공제 (월 1,000 kWh 초과), 저압'] if (fix_usage > 1000) \
            else KEPCO_FEE_STATIC_II['필수사용량 보장공제 (월 200 kWh 이하), 저압']
    deduction = (unit * (fix_usage - 1000) if (fix_usage > 1000) \
            else unit) * -1
    fees.append(deduction)
    print("\t필수사용량 보장공제 : {}원".format(deduction))
fee_dict['필수사용량 보장공제'] = deduction if is_deduction else "-"
    
# 전기요금계
print("전기요금계(기본요금 ＋ 전력량요금 ＋ 기후환경요금 ± 연료비조정액)")
print("-------------------------------------------------")
print("{} 원".format(sum(fees)))
print("-------------------------------------------------")

print("#################################################")

bill_df = bill_df.append(pd.Series(fee_dict, name=household['name']))

#################################################
세대명 : 101 호
기본요금 : 910원
전력량요금 : 13245원
기후환경요금 : 795원
연료비조정액 : -450원
필수사용량 보장공제 여부 : True
	필수사용량 보장공제 : -4000원
전기요금계(기본요금 ＋ 전력량요금 ＋ 기후환경요금 ± 연료비조정액)
-------------------------------------------------
10500 원
-------------------------------------------------
#################################################


In [104]:
# 현재계절 Winter (2월이라 치겠음)
# 세대부 요금 계산
household_bill_df = pd.DataFrame(columns=["사용량 (kWh)","기본요금","전력량요금","기후환경요금","연료비조정액","필수사용량 보장공제",
                               "전기요금계", "부가세", "4사 5입", "전력산업기반기금", "전력산업기반기금 (절사)", 
                                "청구금액", "청구금액 (절사)"])

for household_idx in range(0, len(household_df)):
    household = household_df.iloc[household_idx]
    sel_fee_df = fee_df[
        (fee_df['type'] == "주택용 저압") &\
        (fee_df['season'] == 'winter')
    ].copy()

    fees = list()
    fee_dict = dict()

    # 기본요금
    basic_fee = sel_fee_df[
        sel_fee_df['max kWh'] >= household['usage (kWh)']
    ]['basic'].iloc[0]
    basic_fee
    fees.append(basic_fee)
    fee_dict['기본요금'] = basic_fee

    # 전력량요금
    electricity_rate = 0
    fix_usage = household['usage (kWh)']
    fee_dict["사용량 (kWh)"] = fix_usage
    usage = household['usage (kWh)']
    step_bak = 0
    idx = 0
    while usage > 0:
        progressive_tax = sel_fee_df['fee'].iloc[idx]
        max_kwh = sel_fee_df['max kWh'].iloc[idx]

        step_kwh = usage if usage < max_kwh else max_kwh - step_bak
        electricity_rate += int(progressive_tax * step_kwh)

        usage -= step_kwh
        step_bak = max_kwh
        idx += 1
    fees.append(electricity_rate)
    fee_dict['전력량요금'] = electricity_rate

    # 기후환경요금
    unit = KEPCO_FEE_STATIC['기후환경요금']
    env_fee = int(fix_usage * unit)
    fees.append(env_fee)
    fee_dict['기후환경요금'] = env_fee

    # 연료비조정액
    unit = KEPCO_FEE_STATIC['연료비조정액']
    fuel_cost_adjustment = int(fix_usage * unit)
    fees.append(fuel_cost_adjustment)
    fee_dict['연료비조정액'] = fuel_cost_adjustment

    # 필수사용량 보장공제 여부
    is_deduction = True if (fix_usage > 1000) \
                    or (fix_usage <= 200) else False
    if is_deduction:
        unit = KEPCO_FEE_STATIC['필수사용량 보장공제 (월 1,000 kWh 초과), 저압'] if (fix_usage > 1000) \
                else KEPCO_FEE_STATIC_II['필수사용량 보장공제 (월 200 kWh 이하), 저압']
        deduction = (unit * (fix_usage - 1000) if (fix_usage > 1000) \
                else unit) * -1
        fees.append(deduction)
    fee_dict['필수사용량 보장공제'] = deduction if is_deduction else "-"

    # 전기요금계
    all_fee = sum(fees)
    fee_dict["전기요금계"] = all_fee
    
    VAT = all_fee * 0.1
    VAT_process = round(VAT)
    infra_fund = all_fee * 0.037
    infra_fund_process = math.floor(infra_fund / 10) * 10
    
    fee_dict["부가세"] = VAT
    fee_dict["4사 5입"] = VAT_process
    fee_dict["전력산업기반기금"] = infra_fund
    fee_dict["전력산업기반기금 (절사)"] = infra_fund_process
    
    bill_fee = all_fee + VAT_process + infra_fund_process
    bill_fee_process = math.floor(bill_fee / 10) * 10
    fee_dict["청구금액"] = bill_fee
    fee_dict["청구금액 (절사)"] = bill_fee_process

    household_bill_df = household_bill_df.append(pd.Series(fee_dict, name=household['name']))
household_bill_df

,사용량 (kWh),기본요금,전력량요금,기후환경요금,연료비조정액,필수사용량 보장공제,전기요금계,부가세,4사 5입,전력산업기반기금,전력산업기반기금 (절사),청구금액,청구금액 (절사)
101 호,150.0,910.0,13245.0,795.0,-450.0,-4000.0,10500.0,1050.0,1050.0,388.500,380.0,11930.0,11930.0
201 호,180.0,910.0,15894.0,954.0,-540.0,-4000.0,13218.0,1321.8,1322.0,489.066,480.0,15020.0,15020.0
301 호,220.0,1600.0,21318.0,1166.0,-660.0,-,23424.0,2342.4,2342.0,866.688,860.0,26626.0,26620.0
401 호,210.0,1600.0,19489.0,1113.0,-630.0,-,21572.0,2157.2,2157.0,798.164,790.0,24519.0,24510.0
501 호,310.0,1600.0,37779.0,1643.0,-930.0,-,40092.0,4009.2,4009.0,1483.404,1480.0,45581.0,45580.0
601 호,300.0,1600.0,35950.0,1590.0,-900.0,-,38240.0,3824.0,3824.0,1414.880,1410.0,43474.0,43470.0
701 호,270.0,1600.0,30463.0,1431.0,-810.0,-,32684.0,3268.4,3268.0,1209.308,1200.0,37152.0,37150.0
801 호,190.0,910.0,16777.0,1007.0,-570.0,-4000.0,14124.0,1412.4,1412.0,522.588,520.0,16056.0,16050.0
901 호,250.0,1600.0,26805.0,1325.0,-750.0,-,28980.0,2898.0,2898.0,1072.260,1070.0,32948.0,32940.0
1001 호,260.0,1600.0,28634.0,1378.0,-780.0,-,30832.0,3083.2,3083.0,1140.784,1140.0,35055.0,35050.0


## 공용부
- Season : Winter
- Calc Month : 2
- 요금제 : 일반용전력(갑) 1, 고압A, 선택 2	

In [105]:
APT_meter_kwh = 3000
public_kwh = APT_meter_kwh - household_df['usage (kWh)'].sum()
public_kwh

660

In [106]:
# 요금적용전력, 계약전력
public_bill_df = \
    pd.DataFrame(columns=['최대수요전력 (kW)', '요금적용전력 (kW)', '기본요금',
                         '전력량요금', '기후환경요금', '연료비조정액',
                         '공동사용설비요금',
                         '부가세', '4사 5입',
                         '전력산업기반기금', '전력산업기반기금 (절사)',
                         '청구금액', '청구금액 (절사)'])
peak_calc_month = [1,2,7,8,9,12]
peak_calc_index = list()
for _ in peak_df.index:
    if (_.month in peak_calc_month):
        peak_calc_index.append(_)

fees = list()
fee_dict = dict()
        
maximum_power_demand = peak_df.loc[peak_calc_index]['peak (kW)'].max()
fee_dict['최대수요전력 (kW)'] = maximum_power_demand

charge_applied_power = maximum_power_demand *\
                        (public_kwh / APT_meter_kwh)
fee_dict['요금적용전력 (kW)'] = charge_applied_power

sel_fee_df = fee_df[
    (fee_df['type'] == "일반용전력(갑) 1, 고압A, 선택 2") &\
    (fee_df['season'] == 'winter')
].copy()

# 기본요금
basic_fee = int(charge_applied_power * sel_fee_df['basic'].values[0])
fees.append(basic_fee)
fee_dict['기본요금'] = basic_fee

# 전력량요금
electricity_rate = int(public_kwh * sel_fee_df['fee'].values[0])
fees.append(electricity_rate)
fee_dict['전력량요금'] = electricity_rate

# 기후환경요금
env_fee = int(public_kwh * KEPCO_FEE_STATIC['기후환경요금'])
fees.append(env_fee)
fee_dict['기후환경요금'] = env_fee

# 연료비조정요금
fuel_cost_adjustment = int(public_kwh * KEPCO_FEE_STATIC['연료비조정액'])
fees.append(fuel_cost_adjustment)
fee_dict['연료비조정액'] = fuel_cost_adjustment

all_fee = sum(fees)
fee_dict['공동사용설비요금'] = all_fee

VAT = all_fee * 0.1
VAT_process = int(round(VAT))
infra_fund = all_fee * 0.037
infra_fund_process = int(math.floor(infra_fund / 10) * 10)

bill = all_fee + VAT_process + infra_fund_process
bill_process = int(math.floor(bill / 10) * 10)

fee_dict['부가세'] = VAT
fee_dict['4사 5입'] = VAT_process
fee_dict['전력산업기반기금'] = infra_fund
fee_dict['전력산업기반기금 (절사)'] = infra_fund_process
fee_dict['청구금액'] = bill
fee_dict['청구금액 (절사)'] = bill_process

public_bill_df = public_bill_df.append(
    pd.Series(fee_dict, name="공동사용설비요금 {} 월".format(2)))
public_bill_df

,최대수요전력 (kW),요금적용전력 (kW),기본요금,전력량요금,기후환경요금,연료비조정액,공동사용설비요금,부가세,4사 5입,전력산업기반기금,전력산업기반기금 (절사),청구금액,청구금액 (절사)
공동사용설비요금 2 월,100.0,22.0,181060.0,61578.0,3498.0,-1980.0,244156.0,24415.6,24416.0,9033.772,9030.0,277602.0,277600.0


In [107]:
# common_columns = list(set(household_bill_df.columns) & set(public_bill_df.columns))
common_columns = ['기본요금', '전력량요금', '기후환경요금', '연료비조정액', '부가세', '4사 5입','전력산업기반기금 (절사)','청구금액 (절사)']
bill_df = pd.concat([
    household_bill_df[common_columns],
    public_bill_df[common_columns]
])

management_bill_df = pd.DataFrame(columns=common_columns)
management_dict = dict()

for _ in common_columns:
    management_dict[_] = bill_df[_].sum()

bill_df = bill_df.append(
    pd.Series(
        management_dict,
        name="관리사무소 청구금액"
    )
)
bill_df

,기본요금,전력량요금,기후환경요금,연료비조정액,부가세,4사 5입,전력산업기반기금 (절사),청구금액 (절사)
101 호,910.0,13245.0,795.0,-450.0,1050.0,1050.0,380.0,11930.0
201 호,910.0,15894.0,954.0,-540.0,1321.8,1322.0,480.0,15020.0
301 호,1600.0,21318.0,1166.0,-660.0,2342.4,2342.0,860.0,26620.0
401 호,1600.0,19489.0,1113.0,-630.0,2157.2,2157.0,790.0,24510.0
501 호,1600.0,37779.0,1643.0,-930.0,4009.2,4009.0,1480.0,45580.0
601 호,1600.0,35950.0,1590.0,-900.0,3824.0,3824.0,1410.0,43470.0
701 호,1600.0,30463.0,1431.0,-810.0,3268.4,3268.0,1200.0,37150.0
801 호,910.0,16777.0,1007.0,-570.0,1412.4,1412.0,520.0,16050.0
901 호,1600.0,26805.0,1325.0,-750.0,2898.0,2898.0,1070.0,32940.0
1001 호,1600.0,28634.0,1378.0,-780.0,3083.2,3083.0,1140.0,35050.0


In [108]:
public_bill = bill_df.loc['공동사용설비요금 2 월']['청구금액 (절사)'] / len(household_bill_df)
bill_df['공동전기사용료'] = [public_bill for _
                        in household_bill_df.index] + ["-", "-"]
bill_df['최종청구금액'] = [household_bill_df.loc[_]['청구금액 (절사)'] + public_bill for _
                        in household_bill_df.index] + ["-", "-"]
bill_df

,기본요금,전력량요금,기후환경요금,연료비조정액,부가세,4사 5입,전력산업기반기금 (절사),청구금액 (절사),공동전기사용료,최종청구금액
101 호,910.0,13245.0,795.0,-450.0,1050.0,1050.0,380.0,11930.0,27760.0,39690.0
201 호,910.0,15894.0,954.0,-540.0,1321.8,1322.0,480.0,15020.0,27760.0,42780.0
301 호,1600.0,21318.0,1166.0,-660.0,2342.4,2342.0,860.0,26620.0,27760.0,54380.0
401 호,1600.0,19489.0,1113.0,-630.0,2157.2,2157.0,790.0,24510.0,27760.0,52270.0
501 호,1600.0,37779.0,1643.0,-930.0,4009.2,4009.0,1480.0,45580.0,27760.0,73340.0
601 호,1600.0,35950.0,1590.0,-900.0,3824.0,3824.0,1410.0,43470.0,27760.0,71230.0
701 호,1600.0,30463.0,1431.0,-810.0,3268.4,3268.0,1200.0,37150.0,27760.0,64910.0
801 호,910.0,16777.0,1007.0,-570.0,1412.4,1412.0,520.0,16050.0,27760.0,43810.0
901 호,1600.0,26805.0,1325.0,-750.0,2898.0,2898.0,1070.0,32940.0,27760.0,60700.0
1001 호,1600.0,28634.0,1378.0,-780.0,3083.2,3083.0,1140.0,35050.0,27760.0,62810.0
